##### [< Forrige](10%20-%20statsmodels.ipynb)     |     [Neste >](12%20-%20animasjon%20med%20klasse.ipynb)

# 11 - interaktive websider

## Gapminder

Her er et eksempel på en interaktiv graf som lagres på en nettside. Importere først data:

In [2]:
import pandas as pd
g = pd.read_csv("https://titlon.uit.no/hht/data/gapminder.csv")#reading data

Følgende kode lager en interaktiv graf lik [ett av eksemplene på promosiden til programmet](https://titlon.uit.no/hht/index.php?page=1) med disse dataene:

In [9]:
#!/usr/bin/python
# -*- coding: UTF-8 -*-

from bokeh.plotting import figure, output_file
from bokeh.io import output_notebook, push_notebook, show

import numpy as np




#creating figure:
p = figure(
        title = "", 
        x_axis_type="log",
        x_axis_label='BNP per innbygger (NOK)',
        y_axis_label='Forventet antall leveår',
        tools="hover", 
        tooltips=[
            ("Land","@country"),
            ("BNP/innbygger","@gdp_capNOK"),
            ("Forventet alder","@life_exp_rnd"),
            ("Befolkning","@pop_mill_str")
            ],
        plot_height=580,
        plot_width=980)

#creating a dictionary of continents with norwegian translation:
continents={'Africa': 'Afrika', 'Americas': 'Sør/Nord-Amerika', 'Asia': 'Asia', 'Europe': 'Europa', 'Oceania': 'Oseania'}
#assigning a color hex-code to each continent
colors={'Africa': '#E14827', 'Americas': '#84E127', 'Asia': '#2792E1', 'Europe': '#BC27E1', 'Oceania': '#E04A6C'}

#setting the info displayed when hoovering the plot
g['colors']=g['continent'].apply(lambda cont: colors[cont])
g['size']=g['population'].apply(lambda pop: pop**0.5/300)
g['life_exp_rnd']=g['life_exp'].apply(lambda l: int(l))
g['pop_mill_str']=g['population'].apply(lambda pop:  '{:,}'.format(int(pop/1000000)))
g['gdp_capNOK']=g['gdp_cap'].apply(lambda gdp:  '{:,}'.format(int(8.9*gdp)).replace(',',' '))
g['continent_no']=g['continent'].apply(lambda cont:  continents[cont])

#obtaining a scatterplot:
p.scatter(x = 'gdp_cap', y = 'life_exp', size='size', source=g, color = 'colors', alpha = 0.8,legend_field='continent_no')

x=np.linspace(min(g['gdp_cap']), max(g['gdp_cap']), 100)
p.line(x, res.params['intercept']+res.params['gdp_cap']*np.log(x),color='red')

#formatting:
p.xaxis.major_label_overrides = { 1000: '1k', 10000: '10k', 100000: '100k' }
p.legend.location = "top_left"
p.legend[0].border_line_alpha=0
p.outline_line_alpha=0
p.grid[0].grid_line_alpha=0
p.grid[1].grid_line_alpha=0

#defining output:
output_file('')
output_file("./BNP_levealder.html")
output_notebook()

#drawing it:
show(p)

Loading BokehJS ...

## Interaktivt kart

Interaktivt kart [som vist her](https://titlon.uit.no/hht/index.php?page=2):

In [ ]:
#!/usr/bin/python
# -*- coding: UTF-8 -*-

#Uncomment next two lines to run in goolge colab
#!pip install geopandas
#!pip install geojson


#importing data used to download files:
import zipfile
import urllib
import requests
import io

#importing the geodata and data packages:
import geopandas as gpd
import geojson
import pandas as pd


#folium is the package that creates the map:
import folium


#file locations:
fp_geodata_url = r"https://titlon.uit.no/hht/data/valgkretser.zip"
fp_coins = r"https://titlon.uit.no/hht/img/coins"
fp_data= r"https://titlon.uit.no/hht/data/valgkretser/data.csv"
fp_geodata=r"Valgkrets.geojson"


def main():
    fp_geodata,data,geodata=get_data()
    create_map(fp_geodata,geodata,data)
    
    
def get_data():
    #loading demographic data:
    data=pd.read_csv(fp_data,encoding="latin-1",delimiter=";")  
    
    #loading the geographical data
    #and converting to a coordinate system that folium understands:
    geodata = gpd.read_file(fp_geodata_url)
    geodata=geodata.to_crs('epsg:4326')    
    
    #pandas does not handle æ, ø å, so creates columns in both the demographic data and geodata that can be matched:
    data['Bydel_dec']=data['Bydel'].apply(lambda s: str(s).encode('ascii','ignore').decode('ascii')) 
    geodata['Bydel_dec']=geodata['VKRETS_V_1'].apply(lambda s: s.encode('ascii','ignore').decode('ascii')) 
    
    #saving the geodata to a geoJSON file for use in the map (folium only eats geoJSON)
    geodata.to_file(fp_geodata, driver='GeoJSON')
    
    #returning the path and variable of the geodata, and the demographic data:
    return fp_geodata,geodata, data

def create_map(fp_geodata,data,geodata):
    #creating the map
    tromso=folium.Map(location=[69.67, 18.98],zoom_start=10)
    
    #adding the colour overlay displaying "redness" of municipalities:
    f=folium.Choropleth(
        geo_data=fp_geodata,
        name="choropleth",
        columns=["Bydel_dec", "Rødhet"],
        data=data,
        key_on="feature.properties.Bydel_dec",
        fill_color="RdBu",
        nan_fill_color='white',
        fill_opacity=0.5,
        legend_name="Valgresultat oppslutning H+Frp minus Ap+SV+Rødt",
    ).add_to(tromso)
    
    folium.LayerControl().add_to(tromso)
    
    #the data needs to be merged on a variable with the same name:
    geodata['Bydel']=geodata['VKRETS_V_1']
    
    #merging the demographic data into the geodata:
    geodata=geodata.merge(data,on='Bydel')
    
    
    for i,r in geodata.iterrows():
        #obtaining coordinates:
        lat=float(r['Lat'])
        lon=float(r['Lon'])   
        
        #creating pin for general demographic data 
        t=(f"<b>{r['Bydel']}</b><br>"
            f"Inntekt: {r['Inntekt']}<br>"
            f"Alder: {frmt_int(r['Alder'])}<br>"
            f"Høyere utdanning: {frmt_int(r['Andel med høyere utdanning'])}%<br>"
            f"Ap+SV+Rødt: {frmt_int(r['ApSVRV']*100)}%<br>"
            f"H+FrP: {frmt_int(r['HFrp']*100)}%")
        folium.Marker([lat,lon-0.01],tooltip=t,icon=folium.Icon(color='red')).add_to(tromso)
        
        #creating pin for wealth. The height of the stack of coins is determined by the 
        #picture file f"./img/{int(r['Mynter'])}coins.png":
        icon=folium.features.CustomIcon(f"{fp_coins}/{int(r['Mynter'])}coins.png")
        folium.Marker([lat,lon+0.01],tooltip=f"Formue:{r['Formue']}",icon=icon).add_to(tromso)
        
    #saving the map as a html file:
    tromso.save('./tromso.html')

def frmt_int(value):
    "This function ensures an error free conversion of value to int"
    try:
        return int(value)
    except:
        return value
        

main()

## Oppgaver:

#### Oppgave 1:

Legg til regresjonslinjen fra [10 - statsmodels](10%20-%20statsmodels.ipynb) i den interaktive grafen 

##### [< Forrige](10%20-%20statsmodels.ipynb)     |     [Neste >](12%20-%20animasjon%20med%20klasse.ipynb)